In [ ]:
# X = 제작연도, 매출액, 누적매출액, 누적관객수, 스크린수, 상영횟수, 평균평점
# y = 실제평점

In [1]:
import pandas as pd
df = pd.read_csv('../data/movie_info.csv')
df1 = pd.read_csv('../data/review_rating.csv')

In [2]:
df1.rename({'0':'실제평점', '1':'리뷰', '2':'영화ID', '3':'영화명', '4':'평균평점'}, axis=1, inplace=True)
df1

,실제평점,리뷰,영화ID,영화명,평균평점
0,10,이 영화의 속편은 100% 성공이다. 원래 한국영화들 속편은 잘 안되고 속편 나오면...,0,범죄도시2,8.99
1,10,시리즈로 계속 나왔으면 좋겠다. 마동석한테 최적화된 작품이다.,0,범죄도시2,8.99
2,8,니가 강해상이냐? 아뇨. 구씬데요?,0,범죄도시2,8.99
3,10,전편을 보고 가야 장이수 얼굴만 봐도 웃음이 나오는 이유를 알 수 있습니다:),0,범죄도시2,8.99
4,9,와…. 손석구 연기 진짜 미쳣다…오바하는 범죄자연기가 아니라 ㄹㅇ 범죄자같음,0,범죄도시2,8.99
...,...,...,...,...,...
86136,2,평정사기인듯 진짜너무유치하고 막판진짜 유치b급영화만도못한데 ㅋㅋㅋ요로나얼굴보고무서운...,2999,요로나의 저주,7.73
86137,10,최근 공포영화중 최고!!!,2999,요로나의 저주,7.73
86138,4,"요로나가 무섭긴 한데, 저주를 퍼붓는 이유가 너무 터무니없어서 매력이 느껴지진 않는다.",2999,요로나의 저주,7.73
86139,1,진짜 공포영화 보다가 귀신 응원하기는 처음이다ㅡㅡ 집에 불좀 키고 살아라 이여자야!,2999,요로나의 저주,7.73


In [3]:
df.isnull().sum()

영화ID       0
영화명        0
누적매출액      0
누적관객수      0
스크린수       0
상영횟수       0
개봉일        0
국가         0
장르         0
상영시간      94
감독         0
배우         0
줄거리      310
dtype: int64

In [4]:
df1.dropna(inplace=True)
df1.isnull().sum()

실제평점    0
리뷰      0
영화ID    0
영화명     0
평균평점    0
dtype: int64

In [5]:
# 개봉일에서 제작연도만 빼내기
year_lis = []
for date in df['개봉일']:
    date_lis = []
    date_lis = date.split(".")
    if len(date_lis[0]) > 4:
        date_lis = date_lis[0].split("-")
    year_lis.append(date_lis[0])
df['제작연도'] = year_lis

In [6]:
df = pd.merge(df, df1, how='right', on='영화ID')
df

,영화ID,영화명_x,누적매출액,누적관객수,스크린수,상영횟수,개봉일,국가,장르,상영시간,감독,배우,줄거리,제작연도,실제평점,리뷰,영화명_y,평균평점
0,0,범죄도시2,1.312988e+11,12693415.0,2521.0,355768.0,2022.05.18.,대한민국,"범죄, 액션",106.0,이상용,"마동석,손석구,최귀화,박지환,허동원,하준,정재광,남문철,박지영,이주원,음문석,김찬형...","가리봉동 소탕작전 후 4년 뒤, 금천서 강력반은 베트남으로 도주한 용의자를 인도받아...",2022,10,이 영화의 속편은 100% 성공이다. 원래 한국영화들 속편은 잘 안되고 속편 나오면...,범죄도시2,8.99
1,0,범죄도시2,1.312988e+11,12693415.0,2521.0,355768.0,2022.05.18.,대한민국,"범죄, 액션",106.0,이상용,"마동석,손석구,최귀화,박지환,허동원,하준,정재광,남문철,박지영,이주원,음문석,김찬형...","가리봉동 소탕작전 후 4년 뒤, 금천서 강력반은 베트남으로 도주한 용의자를 인도받아...",2022,10,시리즈로 계속 나왔으면 좋겠다. 마동석한테 최적화된 작품이다.,범죄도시2,8.99
2,0,범죄도시2,1.312988e+11,12693415.0,2521.0,355768.0,2022.05.18.,대한민국,"범죄, 액션",106.0,이상용,"마동석,손석구,최귀화,박지환,허동원,하준,정재광,남문철,박지영,이주원,음문석,김찬형...","가리봉동 소탕작전 후 4년 뒤, 금천서 강력반은 베트남으로 도주한 용의자를 인도받아...",2022,8,니가 강해상이냐? 아뇨. 구씬데요?,범죄도시2,8.99
3,0,범죄도시2,1.312988e+11,12693415.0,2521.0,355768.0,2022.05.18.,대한민국,"범죄, 액션",106.0,이상용,"마동석,손석구,최귀화,박지환,허동원,하준,정재광,남문철,박지영,이주원,음문석,김찬형...","가리봉동 소탕작전 후 4년 뒤, 금천서 강력반은 베트남으로 도주한 용의자를 인도받아...",2022,10,전편을 보고 가야 장이수 얼굴만 봐도 웃음이 나오는 이유를 알 수 있습니다:),범죄도시2,8.99
4,0,범죄도시2,1.312988e+11,12693415.0,2521.0,355768.0,2022.05.18.,대한민국,"범죄, 액션",106.0,이상용,"마동석,손석구,최귀화,박지환,허동원,하준,정재광,남문철,박지영,이주원,음문석,김찬형...","가리봉동 소탕작전 후 4년 뒤, 금천서 강력반은 베트남으로 도주한 용의자를 인도받아...",2022,9,와…. 손석구 연기 진짜 미쳣다…오바하는 범죄자연기가 아니라 ㄹㅇ 범죄자같음,범죄도시2,8.99
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84861,2999,요로나의 저주,1.784935e+09,202756.0,4.0,9.0,2019.04.17.,미국,"공포, 미스터리, 스릴러",93.0,마이클 차베즈,"린다 카델리니,로만 크리스토우,제이니-린 킨첸,레이몬드 크루즈,토니 아멘돌라,패트리...","1973년, 애나는 남편 없이 두 아이와 살고 있다. 사회복지사인 애나가 담당하던 ...",2019,2,평정사기인듯 진짜너무유치하고 막판진짜 유치b급영화만도못한데 ㅋㅋㅋ요로나얼굴보고무서운...,요로나의 저주,7.73
84862,2999,요로나의 저주,1.784935e+09,202756.0,4.0,9.0,2019.04.17.,미국,"공포, 미스터리, 스릴러",93.0,마이클 차베즈,"린다 카델리니,로만 크리스토우,제이니-린 킨첸,레이몬드 크루즈,토니 아멘돌라,패트리...","1973년, 애나는 남편 없이 두 아이와 살고 있다. 사회복지사인 애나가 담당하던 ...",2019,10,최근 공포영화중 최고!!!,요로나의 저주,7.73
84863,2999,요로나의 저주,1.784935e+09,202756.0,4.0,9.0,2019.04.17.,미국,"공포, 미스터리, 스릴러",93.0,마이클 차베즈,"린다 카델리니,로만 크리스토우,제이니-린 킨첸,레이몬드 크루즈,토니 아멘돌라,패트리...","1973년, 애나는 남편 없이 두 아이와 살고 있다. 사회복지사인 애나가 담당하던 ...",2019,4,"요로나가 무섭긴 한데, 저주를 퍼붓는 이유가 너무 터무니없어서 매력이 느껴지진 않는다.",요로나의 저주,7.73
84864,2999,요로나의 저주,1.784935e+09,202756.0,4.0,9.0,2019.04.17.,미국,"공포, 미스터리, 스릴러",93.0,마이클 차베즈,"린다 카델리니,로만 크리스토우,제이니-린 킨첸,레이몬드 크루즈,토니 아멘돌라,패트리...","1973년, 애나는 남편 없이 두 아이와 살고 있다. 사회복지사인 애나가 담당하던 ...",2019,1,진짜 공포영화 보다가 귀신 응원하기는 처음이다ㅡㅡ 집에 불좀 키고 살아라 이여자야!,요로나의 저주,7.73


In [7]:
df.isnull().sum()

영화ID        0
영화명_x     657
누적매출액     657
누적관객수     657
스크린수      657
상영횟수      657
개봉일       657
국가        657
장르        657
상영시간     1039
감독        657
배우        657
줄거리      1138
제작연도      657
실제평점        0
리뷰          0
영화명_y       0
평균평점        0
dtype: int64

In [8]:
df.dropna(subset=['누적매출액'], inplace=True)
df.isnull().sum()

영화ID       0
영화명_x      0
누적매출액      0
누적관객수      0
스크린수       0
상영횟수       0
개봉일        0
국가         0
장르         0
상영시간     382
감독         0
배우         0
줄거리      481
제작연도       0
실제평점       0
리뷰         0
영화명_y      0
평균평점       0
dtype: int64

In [9]:
df = df[['제작연도', '누적매출액', '누적관객수', '스크린수', '상영횟수', '평균평점', '실제평점']]
df

,제작연도,누적매출액,누적관객수,스크린수,상영횟수,평균평점,실제평점
0,2022,1.312988e+11,12693415.0,2521.0,355768.0,8.99,10
1,2022,1.312988e+11,12693415.0,2521.0,355768.0,8.99,10
2,2022,1.312988e+11,12693415.0,2521.0,355768.0,8.99,8
3,2022,1.312988e+11,12693415.0,2521.0,355768.0,8.99,10
4,2022,1.312988e+11,12693415.0,2521.0,355768.0,8.99,9
...,...,...,...,...,...,...,...
84861,2019,1.784935e+09,202756.0,4.0,9.0,7.73,2
84862,2019,1.784935e+09,202756.0,4.0,9.0,7.73,10
84863,2019,1.784935e+09,202756.0,4.0,9.0,7.73,4
84864,2019,1.784935e+09,202756.0,4.0,9.0,7.73,1


In [10]:
X = df[['제작연도', '누적매출액', '누적관객수', '스크린수', '상영횟수', '평균평점']]
y = df['실제평점']

In [11]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0, stratify=y)

In [12]:
# 인공신경망으로 실제 평점 예측하는 모델 생성
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from scikeras.wrappers import KerasClassifier
from keras import models
from keras import layers
def create_network(optimizer="rmsprop"):
    network = models.Sequential()
    network.add(layers.Dense(units=16, activation="relu", input_shape=(X.shape[1],)))
    network.add(layers.Dense(units=16, activation="relu"))
    network.add(layers.Dense(units=1, activation="sigmoid"))
    network.compile(loss='binary_crossentropy', 
                    optimizer=optimizer, # 옵티마이저 
                    metrics=["accuracy"]) # 성능 지표
    return network

In [13]:
neural_network = KerasClassifier(model=create_network, verbose=0)
# 하이퍼파라미터 탐색 영역을 정의
pipe=Pipeline([
    ('scaler', StandardScaler()),
    ('clf', neural_network)
    ])
params={
    'clf__epochs': [50, 100],
    'clf__batch_size': [32,64,128],
    'clf__optimizer': ["rmsprop", "adam"],
    }
grid=GridSearchCV(pipe, params, scoring='accuracy', cv=3)
grid_result = grid.fit(X_train, y_train)

In [ ]:
print(grid_result.best_params_)
print(grid_result.best_score_)

In [ ]:
# 최적의 파라미터로 저장할 수 있는 모델(keras)로 다시 돌리기
def make_model():
    model = models.Sequential()
    model.add(layers.Dense(units=16, activation="relu", input_shape=(X.shape[1],), name="Hidden-1"))
    model.add(layers.Dense(units=16, activation="relu", name="Hidden-2"))
    model.add(layers.Dense(units=1, activation="sigmoid", name="Output"))
    model.compile(loss='binary_crossentropy', 
                    optimizer="adam", # 옵티마이저 
                    metrics=["accuracy"]) # 성능 지표
    return model

In [ ]:
model = make_model()

In [ ]:
# 모델 학습
from keras.callbacks import EarlyStopping
es = EarlyStopping(monitor='val_loss', mode='min', patience=5)
mc = ModelCheckpoint('../model/review_num.h5', monitor='val_accuracy', mode='max', verbose=1, save_best_only=True)
hist = model.fit(X_train, y_train, batch_size = 64, epochs=50, validation_split=0.2, callbacks=[es, mc])

In [ ]:
# 모델 평가
scores = model.evaluate(X, y, verbose=1)
print(scores)
print("%s: %.2f%%" %(model.metrics_names[1], scores[1]*100))

In [ ]:
# csv로 저장
df.to_csv("../data/review_info.csv", index=False)